# Test Atlas v2

See atlas v1 issues:
https://github.com/roocs/clisops/issues/317

In [1]:
import xarray as xr
from clisops.core import subset
from clisops.ops import subset as subset_op

import time
import os

In [2]:
basedir_atlas_v2 = "/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset"

## xarray - CMIP6

TODO: still using two fill values for variable t. Without a fix it is not possible to write as netcdf file.

In [3]:
ds_cmip6 = xr.open_dataset(f"{basedir_atlas_v2}/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc")
ds_cmip6

/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:543: SerializationWarning: variable 't' has multiple fill values {1.0384594e+34, -1.7014118e+38}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


<xarray.Dataset>
Dimensions:          (lat: 180, bnds: 2, lon: 360, time: 1980, member: 30)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) datetime64[ns] 1850-01-01 1850-02-01 ... 2014-12-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

In [4]:
# compression levels

for var in ds_cmip6.data_vars:
    # print(var, ds[var].encoding, "\n")
    complevel = ds_cmip6[var].encoding.get("complevel", 0)
    print(var, "compression level =", complevel)

lat_bnds compression level = 1
lon_bnds compression level = 1
time_bnds compression level = 1
t compression level = 1
crs compression level = 0


In [5]:
# fill values

var_list = list(ds_cmip6.coords) + list(ds_cmip6.data_vars)
for var in var_list:
    fill_value = ds_cmip6[var].encoding.get("_FillValue")
    print(var, "fill value =", fill_value)

lat fill value = None
lon fill value = None
time fill value = None
member_id fill value = None
gcm_institution fill value = None
gcm_model fill value = None
gcm_variant fill value = None
height2m fill value = None
lat_bnds fill value = None
lon_bnds fill value = None
time_bnds fill value = None
t fill value = -1.7014118e+38
crs fill value = None


In [6]:
ds = ds_cmip6.isel(time=0)
ds

<xarray.Dataset>
Dimensions:          (lat: 180, bnds: 2, lon: 360, member: 30)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
    time             datetime64[ns] 1850-01-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (bnds) datetime64[ns] ...
    t                (member, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

In [7]:
try:
    ds.to_netcdf("/tmp/atlas_v2_cmip6.nc")
except ValueError as e:
    print("Fails to write as netcdf!", e)

Fails to write as netcdf! Variable 't' has conflicting _FillValue (-1.7014118346046923e+38) and missing_value (1.0384593717069655e+34). Cannot encode data.


## clisops-core - cmip6

In [8]:
ds = subset.subset_bbox(
    ds_cmip6, lat_bnds=[45, 50], lon_bnds=[-60, -55],
    start_date='2013-01', end_date='2013-12')
ds

<xarray.Dataset>
Dimensions:          (lat: 5, bnds: 2, lon: 5, time: 12, member: 30)
Coordinates:
  * lat              (lat) float64 45.5 46.5 47.5 48.5 49.5
  * lon              (lon) float64 -59.5 -58.5 -57.5 -56.5 -55.5
  * time             (time) datetime64[ns] 2013-01-01 2013-02-01 ... 2013-12-01
    member_id        (member) object 'AS-RCEC_TaiESM1_r1i1p1f1' ... 'NUIST_NE...
    gcm_institution  (member) object 'AS-RCEC' 'AWI' ... 'NOAA-GFDL' 'NUIST'
    gcm_model        (member) object 'TaiESM1' 'AWI-CM-1-1-MR' ... 'NESM3'
    gcm_variant      (member) object 'r1i1p1f1' 'r1i1p1f1' ... 'r1i1p1f1'
    height2m         float64 2.0
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

In [9]:
try:
    ds.to_netcdf("/tmp/atlas_v2_cmip6.nc")
except ValueError as e:
    print("Fails to write as netcdf!", e)

Fails to write as netcdf! Variable 't' has conflicting _FillValue (-1.7014118346046923e+38) and missing_value (1.0384593717069655e+34). Cannot encode data.


## clisops-ops - cmip6

In [10]:
start = time.time()

outputs = subset_op(
    ds=ds_cmip6,
    time="2013-01/2013-01",
    # area=(0.0, 10.0, 175.0, 90.0),
    output_type="xarray",
)

duration = time.time() - start
print(f"duration: {duration} secs")

outputs[0]

duration: 0.014023065567016602 secs


<xarray.Dataset>
Dimensions:          (lat: 180, bnds: 2, lon: 360, time: 24, member: 30)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) datetime64[ns] 2013-01-01 2013-02-01 ... 2014-12-01
    member_id        (member) object 'AS-RCEC_TaiESM1_r1i1p1f1' ... 'NUIST_NE...
    gcm_institution  (member) object 'AS-RCEC' 'AWI' ... 'NOAA-GFDL' 'NUIST'
    gcm_model        (member) object 'TaiESM1' 'AWI-CM-1-1-MR' ... 'NESM3'
    gcm_variant      (member) object 'r1i1p1f1' 'r1i1p1f1' ... 'r1i1p1f1'
    height2m         float64 2.0
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

In [11]:
! rm /tmp/output_*

In [12]:
start = time.time()

outputs = subset_op(
    ds=ds_cmip6,
    time="2013-01/2013-01",
    # area=(0.0, 10.0, 175.0, 90.0),
    output_type="nc",
    output_dir="/tmp",
    split_method="time:auto",
    file_namer="simple"
)

duration = time.time() - start
print(f"duration: {duration} secs")

print("Number of output files: ", len(outputs))
outputs[0]

/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable member_id has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable gcm_institution has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable gcm_model has data in the form of a dask array with dtype=object, which means it is being loaded int

duration: 11.206794261932373 secs
Number of output files:  1


'/tmp/output_001.nc'

In [13]:
file_size = os.path.getsize(outputs[0])
print("File Size is :", file_size/(1024*1024), "MB")

File Size is : 71.30526638031006 MB


In [14]:
ds = xr.open_dataset(outputs[0])
ds

<xarray.Dataset>
Dimensions:          (lat: 180, bnds: 2, lon: 360, time: 24, member: 30)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) datetime64[ns] 2013-01-01 2013-02-01 ... 2014-12-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

In [15]:
ds.to_netcdf("/tmp/atlas_v2_cmip6.nc")